<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/moduler_optimalisering_evaulering/Moduler_optimalisering_evaluering_veiledet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotek

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
import time

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean, stdev

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


import optuna
from optuna.visualization import plot_optimization_history

# Egendefinerte Moduler

In [ ]:
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import evaluering_beregning
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval

Evalueringsmetrikker

In [ ]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_sann, y_pred):

  return np.sqrt(mean_squared_error(y_sann, y_pred))

# Moduler for optimalisering etter treningsandel etter algoritme

## Modelltrening etter enzymtype

### KNeighborsRegressor

In [ ]:
def trening_knr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      testsett,
                      n_splitt,
                      n_gjentagelser,
                      n_runder,
                      random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Deler opp i forklaring og respons variabler
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 10),
          'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
          'p': trial.suggest_int('p', 1, 10)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])

    # Kryssvalidering
    rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                   n_repeats=n_gjentagelser,
                                   random_state=random_seed)

    # Lagring av RMSE resultater
    rmse_validering_resultat = []

    step = 0
    # Går igjennom folds i kryssvalidering
    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      # Forklaringsvaribel folds
      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      # Responsvariabel folds
      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      # Modelltrening
      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      # Prediksjon
      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      # Evaluering: RMSE
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step >= math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    # RMSE snittscore
    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modeloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=10,
                                  reduction_factor=4))
  study.optimize(objective, n_trials=n_runder, n_jobs=2)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  knr_pipeline.fit(X_trening_markert, y_trening_markert)

  # Prediksjoner
  y_test_pred = knr_pipeline.predict(X_test)
  y_trening_pred = knr_pipeline.predict(X_trening_markert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  return resultater, beste_parametere, rmse_score_opt

### RandomForestRegressor

In [ ]:
def trening_rfr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      testsett,
                      n_splitt,
                      n_gjentagelser,
                      n_runder,
                      random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Deler opp i forklaring og respons variabler
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # RFR
  def objective(trial):

    # Initierer parameter-grid
    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 500),
          'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']),
          'max_depth': trial.suggest_int('max_depth', 1, 30),
          'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
          'max_features': trial.suggest_float('max_features', 0.1, 1.0, step=0.001)
      }

    # Initierer modell med skalering
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           n_jobs=2,
                                           random_state=random_seed))
      ])

    # Kryssvalidering
    rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                   n_repeats=n_gjentagelser,
                                   random_state=random_seed)

    # Lagring av RMSE resultater
    rmse_validering_resultat = []

    step = 0
    # Går igjennom folds i kryssvalidering
    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      # Forklaringsvaribel folds
      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      # Responsvariabel folds
      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      # Modelltrening
      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      # Prediksjon
      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)

      # Evaluering: RMSE
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step >= math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    # RMSE snittscore
    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modelloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=10,
                                  reduction_factor=4))
  study.optimize(objective, n_trials=n_runder, n_jobs=2)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                      random_state=random_seed))
  ])

  # Trener modell for videre evaluering

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  rfr_pipeline.fit(X_trening_markert, y_trening_markert)

  # Prediksjoner
  y_test_pred = rfr_pipeline.predict(X_test)
  y_trening_pred = rfr_pipeline.predict(X_trening_markert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  return resultater, beste_parametere, rmse_score_opt

### Support Vector Regressor

In [ ]:
def trening_svr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      testsett,
                      n_splitt,
                      n_gjentagelser,
                      n_runder,
                      random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Deler opp i forklaring og respons variabler
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # SVR
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'degree': trial.suggest_int('degree', 1, 20),
          'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
          'C': trial.suggest_float('C', 0.01, 100, step=0.001),
          'epsilon': trial.suggest_float('epsilon', 0.01, 100, step=0.001),
      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])

    # Kryssvalidering
    rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                   n_repeats=n_gjentagelser,
                                   random_state=random_seed)

    # Lagring av RMSE resultater
    rmse_validering_resultat = []

    step = 0
    # Går igjennom folds i kryssvalidering
    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_treningssett_markert):

      # Forklaringsvaribel folds
      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      # Responsvariabel folds
      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      # Modelltrening
      svr_pipeline.fit(X_trening_fold, y_trening_fold)

      # Prediksjon
      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)

      # Evaluering: RMSE
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step >= math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    # RMSE snittscore
    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modeloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=10,
                                  reduction_factor=4))
  study.optimize(objective, n_trials=n_runder, n_jobs=2)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  svr_pipeline.fit(X_trening_markert, y_trening_markert)

  # Prediksjoner
  y_test_pred = svr_pipeline.predict(X_test)
  y_trening_pred = svr_pipeline.predict(X_trening_markert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  return resultater, beste_parametere, rmse_score_opt

## Modelltrening etter dag og kontinuitet

### KNeighborsRegressor

In [ ]:
def trening_knr_dag(rå_data,
                    treningssett_markert,
                    testsett,
                    n_splitt,
                    n_runder,
                    random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Deler opp i forklaring og respons variabler
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # KNR
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 10),
          'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
          'p': trial.suggest_int('p', 1, 10)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])


    # Kryssvalidering
    treningssett, valideringssett\
    = trening_validering_oppdeling_dag(rå_data,
                                       treningssett_markert,
                                       n_splitt=n_splitt,
                                       test_ekskluder_kalkun_kylling=True)

    # Lagring av RMSE resultater
    rmse_validering_resultat = []

    step = 0
    # Går igjennom folds i kryssvalidering
    for i, j in zip(treningssett, valideringssett):

      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (treningssett_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.loc[val, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)
      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step >= math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modeloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=10,
                                  reduction_factor=4))
  study.optimize(objective, n_trials=n_runder, n_jobs=2)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  knr_pipeline.fit(X_trening_markert, y_trening_markert)

  # Prediksjoner
  y_test_pred = knr_pipeline.predict(X_test)
  y_trening_pred = knr_pipeline.predict(X_trening_markert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  return resultater, beste_parametere, rmse_score_opt

### RandomForestRegressor

In [ ]:
def trening_rfr_dag(rå_data,
                    treningssett_markert,
                    testsett,
                    n_splitt,
                    n_runder,
                    random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Deler opp i forklaring og respons variabler
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # RFR
  def objective(trial):

    # Initierer parameter-grid
    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 500),
          'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']),
          'max_depth': trial.suggest_int('max_depth', 1, 30),
          'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
          'max_features': trial.suggest_float('max_features', 0.1, 1.0, step=0.001)
      }

    # Initierer modell med skalering
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           n_jobs=2,
                                           random_state=random_seed))
      ])

    # Kryssvalidering
    treningssett, valideringssett\
    = trening_validering_oppdeling_dag(rå_data,
                                       treningssett_markert,
                                       n_splitt=n_splitt,
                                       test_ekskluder_kalkun_kylling=True)

    # Lagring av RMSE resultater
    rmse_validering_resultat = []

    step = 0
    # Går igjennom folds i kryssvalidering
    for i, j in zip(treningssett, valideringssett):

      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (treningssett_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.loc[val, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)
      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step >= math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modeloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=10,
                                  reduction_factor=4))
  study.optimize(objective, n_trials=n_runder, n_jobs=2)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                      random_state=random_seed))
  ])

  # Trener modell for videre evaluering

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  rfr_pipeline.fit(X_trening_markert, y_trening_markert)

  # Prediksjoner
  y_test_pred = rfr_pipeline.predict(X_test)
  y_trening_pred = rfr_pipeline.predict(X_trening_markert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  return resultater, beste_parametere, rmse_score_opt

### Support Vector Regressor

In [ ]:
def trening_svr_dag(rå_data,
                    treningssett_markert,
                    testsett,
                    n_splitt,
                    n_runder,
                    random_seed=2024):

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Deler opp i forklaring og respons variabler
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]

  # SVR
  def objective(trial):

    parametere = {
          'kernel': trial.suggest_categorical('kernel',["linear", "poly", "rbf"]),
          'degree': trial.suggest_int('degree', 1, 20),
          'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
          'C': trial.suggest_float('C', 0.01, 100, step=0.001),
          'epsilon': trial.suggest_float('epsilon', 0.01, 100, step=0.001),
      }

    svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**parametere))
      ])


    # Kryssvalidering
    treningssett, valideringssett\
    = trening_validering_oppdeling_dag(rå_data,
                                       treningssett_markert,
                                       n_splitt=n_splitt,
                                       test_ekskluder_kalkun_kylling=True)

    # Lagring av RMSE resultater
    rmse_validering_resultat = []

    step = 0
    # Går igjennom folds i kryssvalidering
    for i, j in zip(treningssett, valideringssett):

      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (treningssett_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.loc[val, :]).iloc[:, -1]

      svr_pipeline.fit(X_trening_fold, y_trening_fold)
      y_validering_prediksjon = svr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      # Tidlig stopp
      if step >= math.floor(n_splitt/2):
        rmse_val_snitt_midlertidig = mean(rmse_validering_resultat)
        trial.report(rmse_val_snitt_midlertidig, step)
        if trial.should_prune():
          raise optuna.TrialPruned()
      step += 1

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  # Optuna - Modeloptimalisering

  # Lagrer tid for optimalisering
  start_tid_opt = time.time()

  study = optuna.create_study(direction='minimize',
                              pruner=optuna.pruners.HyperbandPruner(
                                  min_resource=10,
                                  reduction_factor=4))

  study.optimize(objective, n_trials=n_runder, n_jobs=2)

  tid_opt_sek= time.time() - start_tid_opt
  tid_opt = time.strftime("%H:%M:%S", time.gmtime(tid_opt_sek))

  # Henter val RMSE snitt score og parametere fra studiet
  rmse_score_opt = study.best_value
  beste_parametere = study.best_params

  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**beste_parametere))
      ])

  # Trener modell for videre evaluering

  # Lagrer tid for modelltrening
  start_tid_modelltrening = time.time()

  svr_pipeline.fit(X_trening_markert, y_trening_markert)

  # Prediksjoner
  y_test_pred = svr_pipeline.predict(X_test)
  y_trening_pred = svr_pipeline.predict(X_trening_markert)

  tid_trening_sek = time.time() - start_tid_modelltrening
  tid_trening = time.strftime("%H:%M:%S", time.gmtime(tid_trening_sek))

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  = evaluering_beregning(y_test,
                         y_test_pred)

  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_trening_markert,
                         y_trening_pred)

  # Legger resulater til i datasett
  resultater = [rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                trening_markert_antall, tid_trening,
                n_runder, n_splitt, tid_opt]

  return resultater, beste_parametere, rmse_score_opt

# Modellevalusering etter enzymtype

In [ ]:
def veiledet_modell_evaluering_1(dict_res_opt: dict,
                                  dict_res: dict,
                                  treningssett_markert: pd.DataFrame,
                                  enzymtyper_treningssett_markert: pd.Series,
                                  testsett: pd.DataFrame,
                                  n_splitt: int,
                                  n_gjentagelser: int,
                                  n_runder: int,
                                  test_andel: float,
                                  trenings_andel: float,
                                  random_seed: int = 2024
                                  ):





  """
  Funksjon som evaluerer veilede modeller: RFR, KNR, SVR.
  Oppdeling i trening, validering og testing har hensyn til
  Alternativ 1: Enzymfordeling.

  Args:
      - dict_res_opt (dict): For lagringa av resultater fra modelloptimalisering.
      - dict_res (dict): For lagring av resultater fra modellevaluering.
      - treningssett_markert (pd.DataFrame): Datasett med merket data.
      - enzymtyper_markert_datasett (pd.Series): Enzymtyper for merkedet data.
      - testsett (pd.DataFrame): Testsett.
      - n_splitt (int): Antall folds for trenings- og valideringssett.
      - n_gjentagelser (int): Antall gjentelser i kryssevalidering.
      - n_runder (int): Antall runder for optimalisering av modellparameterne.
      - test_andel (float): Andel av datae som brukes til testing.
      - trenings_andel (float): Andel av markert treningsdata som brukes til modelltrening.
      - random_seed (int): Tilfeldighetsfrø.
  """


  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]



  # Modelloptimalisering - KNR
  knr_resultater, knr_parametere, knr_opt_rmse\
  = trening_knr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      testsett,
                      n_splitt,
                      n_gjentagelser,
                      n_runder,
                      random_seed)

  # Lagring av informasjon om parametere for treningsandel
  res_eval_opt(dict_res_opt, 'KNR', test_andel,
                trenings_andel, knr_parametere, knr_opt_rmse, knr_resultater)

  # KNR - beste modell
  # Initierer modell
  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**knr_parametere))
      ])

  # Modelloptimalisering - RFR
  rfr_resultater, rfr_parametere, rfr_opt_rmse\
  = trening_rfr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      testsett,
                      n_splitt,
                      n_gjentagelser,
                      n_runder,
                      random_seed)

  # Lagring av informasjon om parametere for treningsandel
  res_eval_opt(dict_res_opt, 'RFR', test_andel,
                trenings_andel, rfr_parametere, rfr_opt_rmse, rfr_resultater)

  # RFR - beste modell
  # Initierer modell
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**rfr_parametere,
                                        random_state=random_seed))
  ])

  # Modelloptimalisering - SVR
  svr_resultater, svr_parametere, svr_opt_rmse\
  = trening_svr_enzym(treningssett_markert,
                      enzymtyper_treningssett_markert,
                      testsett,
                      n_splitt,
                      n_gjentagelser,
                      n_runder,
                      random_seed)

  # Lagring av informasjon om parametere for treningsandel
  res_eval_opt(dict_res_opt, 'SVR', test_andel,
                trenings_andel, svr_parametere, svr_opt_rmse, svr_resultater)

  # SVR - beste modell
  # Initierer modell
  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**svr_parametere))
      ])

  # Lagring av resultater fra kryssvalidering
  knr_res = Resultat_evaluering()
  rfr_res = Resultat_evaluering()
  svr_res = Resultat_evaluering()

  # Kryssvalidering
  rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                  n_repeats=n_gjentagelser,
                                  random_state=random_seed)

  # Går igjennom kryssvalidering
  for train, val in rskf.split(treningssett_markert,
                                enzymtyper_treningssett_markert):

    X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
    X_val_fold = (treningssett_markert.iloc[val, :]).iloc[:, :-1]

    y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
    y_val_fold = (treningssett_markert.iloc[val, :]).iloc[:, -1]


    #KNR
    knr_pipeline.fit(X_trening_fold, y_trening_fold)

    # Prediksjoner
    y_trening_pred = knr_pipeline.predict(X_trening_fold)
    y_val_pred = knr_pipeline.predict(X_val_fold)
    y_test_pred = knr_pipeline.predict(X_test)

    # Evaluering
    knr_res.evaluering(y_trening_fold,
                        y_trening_pred,
                        y_val_fold,
                        y_val_pred,
                        y_test,
                        y_test_pred)

    #RFR
    rfr_pipeline.fit(X_trening_fold, y_trening_fold)

    # Prediksjoner
    y_trening_pred = rfr_pipeline.predict(X_trening_fold)
    y_val_pred = rfr_pipeline.predict(X_val_fold)
    y_test_pred = rfr_pipeline.predict(X_test)

    # Evaluering
    rfr_res.evaluering(y_trening_fold,
                        y_trening_pred,
                        y_val_fold,
                        y_val_pred,
                        y_test,
                        y_test_pred)


    #SVR
    svr_pipeline.fit(X_trening_fold, y_trening_fold)

    # Prediksjoner
    y_trening_pred = svr_pipeline.predict(X_trening_fold)
    y_val_pred = svr_pipeline.predict(X_val_fold)
    y_test_pred = svr_pipeline.predict(X_test)

    # Evaluering
    svr_res.evaluering(y_trening_fold,
                        y_trening_pred,
                        y_val_fold,
                        y_val_pred,
                        y_test,
                        y_test_pred)

  # Lagring av informasjon om paramtere for treningsandel

  res_eval(dict_res, 'KNR', test_andel,
          trenings_andel, trening_markert_antall,
          parametere=knr_parametere,
          rmse_score_opt=knr_opt_rmse,
          n_splitt=n_splitt,
          res_eval=knr_res)

  res_eval(dict_res, 'RFR', test_andel,
            trenings_andel, trening_markert_antall,
            parametere=rfr_parametere,
            rmse_score_opt=rfr_opt_rmse,
            n_splitt=n_splitt,
            res_eval=rfr_res)

  res_eval(dict_res, 'SVR', test_andel,
            trenings_andel, trening_markert_antall,
            parametere=svr_parametere,
            rmse_score_opt=svr_opt_rmse,
            n_splitt=n_splitt,
            res_eval=svr_res)

# Modelevaluering etter dag og kontinuitet

In [ ]:
def veiledet_modell_evaluering_2(dict_res_opt: dict,
                                 dict_res: dict,
                                 rå_data: pd.DataFrame,
                                 treningssett_markert: pd.DataFrame,
                                 testsett: pd.DataFrame,
                                 n_splitt: int,
                                 n_runder: int,
                                 test_andel: float,
                                 trenings_andel: float,
                                 random_seed: int = 2024
                                 ):




  """
  Funksjon som evaluerer veilede modeller: RFR, KNR, SVR.
  Oppdeling i trening, validering og testing har hensyn til
  Alternativ 2: Dag og kontinuitet.

  Args:
      - dict_res_opt (dict): For lagringa av resultater fra modelloptimalisering.
      - dict_res (dict): For lagring av resultater fra modellevaluering.
      - rå_data (pd.DataFrame): Original data med alle observasjoner og egenskaper
      - treningssett_markert (pd.DataFrame): Datasett med merket data.
      - testsett (pd.DataFrame): Testsett.
      - n_splitt (int): Antall folds for trenings- og valideringssett.
      - n_runder (int): Antall runder for optimalisering av modellparameterne.
      - test_andel (float): Andel av datae som brukes til testing.
      - trenings_andel (float): Andel av markert treningsdata som brukes til modelltrening.
      - random_seed (int): Tilfeldighetsfrø.
  """

  # Antall markert data til trening
  trening_markert_antall = treningssett_markert.shape[0]

  # Testsett
  X_test = testsett.iloc[:, :-1]
  y_test = testsett.iloc[:, -1]



  # Modelloptimalisering - KNR
  knr_resultater, knr_parametere, knr_opt_rmse\
  = trening_knr_dag(rå_data,
                    treningssett_markert,
                    testsett,
                    n_splitt,
                    n_runder,
                    random_seed)

  # Lagring av informasjon om parametere for treningsandel
  res_eval_opt(dict_res_opt, 'KNR', test_andel,
               trenings_andel, knr_parametere, knr_opt_rmse, knr_resultater)

  # KNR - beste modell
  # Initierer modell
  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**knr_parametere))
      ])


  # Modelloptimalisering - RFR
  rfr_resultater, rfr_parametere, rfr_opt_rmse\
  = trening_rfr_dag(rå_data,
                    treningssett_markert,
                    testsett,
                    n_splitt,
                    n_runder,
                    random_seed)

  # Lagring av informasjon om parametere for treningsandel
  res_eval_opt(dict_res_opt, 'RFR', test_andel,
               trenings_andel, rfr_parametere, rfr_opt_rmse, rfr_resultater)

  # RFR - beste modell
  # Initierer modell
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**rfr_parametere,
                                       random_state=random_seed))
  ])



  # Modelloptimalisering - SVR
  svr_resultater, svr_parametere, svr_opt_rmse\
  = trening_svr_dag(rå_data,
                    treningssett_markert,
                    testsett,
                    n_splitt,
                    n_runder,
                    random_seed)

  # Lagring av informasjon om parametere for treningsandel
  res_eval_opt(dict_res_opt, 'SVR', test_andel,
               trenings_andel, svr_parametere, svr_opt_rmse, svr_resultater)

  # SVR - beste modell
  # Initierer modell
  svr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', SVR(**svr_parametere))
      ])

  # Lagring av resultater fra kryssvalidering
  knr_res = Resultat_evaluering()
  rfr_res = Resultat_evaluering()
  svr_res = Resultat_evaluering()


  # Kryssvalidering
  trenings_sett, val_sett\
  = trening_validering_oppdeling_dag(rå_data,
                                     treningssett_markert,
                                     n_splitt=n_splitt,
                                     test_ekskluder_kalkun_kylling=True)

  for i, j in zip(trenings_sett, val_sett):
    train = trenings_sett[i]
    val = val_sett[j]

    X_trening_fold = (treningssett_markert.loc[train, :]).iloc[:, :-1]
    X_val_fold = (treningssett_markert.loc[val, :]).iloc[:, :-1]

    y_trening_fold = (treningssett_markert.loc[train, :]).iloc[:, -1]
    y_val_fold = (treningssett_markert.loc[val, :]).iloc[:, -1]

    #KNR
    knr_pipeline.fit(X_trening_fold, y_trening_fold)

    # Prediksjoner
    y_trening_pred = knr_pipeline.predict(X_trening_fold)
    y_val_pred = knr_pipeline.predict(X_val_fold)
    y_test_pred = knr_pipeline.predict(X_test)

    # Evaluering
    knr_res.evaluering(y_trening_fold,
                       y_trening_pred,
                       y_val_fold,
                       y_val_pred,
                       y_test,
                       y_test_pred)

    #RFR
    rfr_pipeline.fit(X_trening_fold, y_trening_fold)

    # Prediksjoner
    y_trening_pred = rfr_pipeline.predict(X_trening_fold)
    y_val_pred = rfr_pipeline.predict(X_val_fold)
    y_test_pred = rfr_pipeline.predict(X_test)

    # Evaluering
    rfr_res.evaluering(y_trening_fold,
                       y_trening_pred,
                       y_val_fold,
                       y_val_pred,
                       y_test,
                       y_test_pred)

    #SVR
    svr_pipeline.fit(X_trening_fold, y_trening_fold)

    # Prediksjoner
    y_trening_pred = svr_pipeline.predict(X_trening_fold)
    y_val_pred = svr_pipeline.predict(X_val_fold)
    y_test_pred = svr_pipeline.predict(X_test)

    # Evaluering
    svr_res.evaluering(y_trening_fold,
                       y_trening_pred,
                       y_val_fold,
                       y_val_pred,
                       y_test,
                       y_test_pred)

  # Lagring av informasjon om paramtere for treningsandel

  res_eval(dict_res, 'KNR', test_andel,
          trenings_andel, trening_markert_antall,
          parametere=knr_parametere,
          rmse_score_opt=knr_opt_rmse,
          n_splitt=n_splitt,
          res_eval=knr_res)

  res_eval(dict_res, 'RFR', test_andel,
           trenings_andel, trening_markert_antall,
           parametere=rfr_parametere,
           rmse_score_opt=rfr_opt_rmse,
           n_splitt=n_splitt,
           res_eval=rfr_res)

  res_eval(dict_res, 'SVR', test_andel,
           trenings_andel, trening_markert_antall,
           parametere=svr_parametere,
           rmse_score_opt=svr_opt_rmse,
           n_splitt=n_splitt,
           res_eval=svr_res)